In [1]:
# !pip install --upgrade gensim
#!pip install scikeras[tensorflow]

In [2]:
import pandas as pd
from functions import utils, RNN
from sklearn.model_selection import train_test_split, GridSearchCV
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.pipeline import Pipeline

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [131]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import GridSearchCV
from gensim.models import Word2Vec
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold

corpus = api.load('text8')
vocab_size = 100
max_length = None
vector_size = 300
tokenizer = Tokenizer()
label_encoder = LabelEncoder()

def preprocess_split(X_train, X_test, y_train, y_test):
    global max_length
    global vocab_size
    global vector_size
    #Encode the sentiment labels to 0 and n-1 classes
    tokenizer.fit_on_texts(X_train)
    
    X_train_seq = tokenizer.texts_to_sequences(X_train)
    X_test_seq = tokenizer.texts_to_sequences(X_test)
    
    vocab_size = len(tokenizer.word_index) + 1
    max_length = max(len(seq) for seq in X_train_seq)
    
    print("MAX LENGTH: ", max_length, vector_size)
    
    # Train Word2Vec model
#     word2vec_model = api.load("word2vec-google-news-300")# Word2Vec(sentences=corpus, vector_size=vector_size, window=5, min_count=1, workers=4)
    word2vec_model = Word2Vec(sentences=corpus, vector_size=vector_size, window=5, min_count=1, workers=4)
    vector_size = word2vec_model.vector_size
    
    # Convert text sequences to word vectors
    X_train_vec = convert_text_to_vec(word2vec_model, X_train_seq, max_length)
    X_test_vec = convert_text_to_vec(word2vec_model, X_test_seq, max_length)
    
    return X_train_vec, X_test_vec, y_train, y_test

def train_model(df):
    global max_length
    global vocab_size
    global vector_size
    
    X = df['cleaned_text']
    y = df['sentiment_label']
    y = label_encoder.fit_transform(y)
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    X_train_vec, X_test_vec, y_train, y_test = preprocess_split(X_train, X_test, y_train, y_test)
    rnn_model = KerasClassifier(build_fn=create_rnn_model, epochs=10, batch_size=32)

    # Fit the model
    rnn_model.fit(X_train_vec, y_train)

    # Evaluate the model
    accuracy = rnn_model.score(X_test_vec, y_test)
    
    print("Accuracy: ", accuracy)
    
    return rnn_model, X_train_vec, X_test_vec, y_train, y_test

def create_rnn_model():
    model = Sequential()
    model.add(LSTM(units=128, input_shape=(max_length, vector_size)))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def convert_text_to_vec(word2vec_model, X_train_seq, max_length):
    X_vec = []
    for seq in X_train_seq:
        seq_vec = []
        for word_id in seq:
            try:
                word = tokenizer.index_word[word_id]
                word_vector = word2vec_model.wv[word]
#                 print(word, word_vector)
            except KeyError:
                word_vector = np.zeros(vector_size)
            seq_vec.append(word_vector)
        X_vec.append(seq_vec)
        
    X_vec = pad_sequences(X_vec, maxlen=max_length, padding="post", dtype='float32')
    X_vec = X_vec.reshape(X_vec.shape[0], X_vec.shape[1], -1)
    return X_vec

def hypertune(model, X_train_vec, y_train, X_test_vec, y_test):
    parameters = {'epochs': [10, 20, 30], 'batch_size': [32, 64, 128]}
    grid_search = GridSearchCV(estimator=model, param_grid=parameters, scoring='accuracy', cv=5)
    grid_search.fit(X_train_vec, y_train)
    return grid_search

def cross_validate(df, model):
    # Define the number of folds for cross-validation
    n_splits = 5
    X = df['cleaned_text']
    y = df['sentiment_label']
    # Initialize the StratifiedKFold object
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True)

    # Perform cross-validation
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        X_train_vec, X_test_vec, y_train, y_test = preprocess_split(X_train, X_test, y_train, y_test)
        
        # Train and evaluate the model on the current fold
        model.fit(X_train_vec, y_train)
        accuracy = model.score(X_test_vec, y_test)

        print(f"Accuracy: {accuracy}")

In [58]:
irelandNewsDF = pd.read_csv('datasets/ie_news_cleaned.csv')
ukNewsDF = pd.read_csv('datasets/uk_news_cleaned.csv')

In [59]:
irelandNewsDF.head()

,Unnamed: 0,title,publish_date,source_country,sentiment,cleaned_text,sentiment_label,word_length,char_length,month
0,1,Ministers Harris and Ryan welcome record numbe...,2023-01-17 21:41:06,ie,0.341,minister harris ryan welcome record number enr...,positive,16,113,January 2023
1,2,New €4.8m state-of-the-art facility planned fo...,2022-12-05 11:06:50,ie,0.377,new 4.8m state-of-the-art facility planned ucd...,positive,9,62,December 2022
2,3,‘Not possible’ to make Northern Ireland Protoc...,2023-02-01 20:45:45,ie,-0.146,possible make northern ireland protocol work e...,negative,12,69,February 2023
3,4,Shoppers must demand longer lasting goods and ...,2023-01-10 16:14:04,ie,-0.077,shopper must demand longer lasting good le pac...,negative,15,95,January 2023
4,5,Opportunity knocks for investors willing to ma...,2022-12-06 07:00:00,ie,-0.137,opportunity knock investor willing make long-t...,negative,13,90,December 2022


In [60]:
irelandNewsDF['sentiment_label'].shape

(426,)

RNN Model

In [105]:
# model, X_train_padded, X_test_padded, y_train, y_test, y_pred = RNN.train_model(irelandNewsDF)
model, X_train, X_test, y_train, y_test = train_model(irelandNewsDF)

MAX LENGTH:  21 300
Epoch 1/10


C:\Users\asus\AppData\Local\Temp/ipykernel_27504/3650679903.py:59: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  rnn_model = KerasClassifier(build_fn=create_rnn_model, epochs=10, batch_size=32)


11/11 [==============================] - 1s 13ms/step - loss: 0.4618 - accuracy: 0.0147
Epoch 2/10
11/11 [==============================] - 0s 13ms/step - loss: -0.0052 - accuracy: 0.0029
Epoch 3/10
11/11 [==============================] - 0s 14ms/step - loss: -0.7451 - accuracy: 0.0706
Epoch 4/10
11/11 [==============================] - 0s 14ms/step - loss: -2.0389 - accuracy: 0.2412
Epoch 5/10
11/11 [==============================] - 0s 14ms/step - loss: -3.1359 - accuracy: 0.3441
Epoch 6/10
11/11 [==============================] - 0s 14ms/step - loss: -4.0359 - accuracy: 0.3265
Epoch 7/10
11/11 [==============================] - 0s 14ms/step - loss: -4.3790 - accuracy: 0.3206
Epoch 8/10
11/11 [==============================] - 0s 14ms/step - loss: -5.7342 - accuracy: 0.3559
Epoch 9/10
11/11 [==============================] - 0s 14ms/step - loss: -6.4500 - accuracy: 0.3265
Epoch 10/10
3/3 [==============================] - 0s 5ms/step - loss: 1.9561 - accuracy: 0.2326
Accuracy:  0.23

In [106]:
ie_RNN_grid_search = hypertune(model, X_train_vec=X_train, y_train=y_train, y_test=y_test, X_test_vec=X_test)

Epoch 1/10
9/9 [==============================] - 1s 13ms/step - loss: 0.4511 - accuracy: 0.0478
Epoch 2/10
9/9 [==============================] - 0s 13ms/step - loss: -0.1034 - accuracy: 0.0037
Epoch 3/10
9/9 [==============================] - 0s 13ms/step - loss: -0.5267 - accuracy: 0.0221
Epoch 4/10
9/9 [==============================] - 0s 13ms/step - loss: -1.8587 - accuracy: 0.2132
Epoch 5/10
9/9 [==============================] - 0s 14ms/step - loss: -2.0007 - accuracy: 0.1287
Epoch 6/10
9/9 [==============================] - 0s 14ms/step - loss: -3.3343 - accuracy: 0.3603
Epoch 7/10
9/9 [==============================] - 0s 14ms/step - loss: -4.2579 - accuracy: 0.2941
Epoch 8/10
9/9 [==============================] - 0s 14ms/step - loss: -4.8489 - accuracy: 0.2500
Epoch 9/10
9/9 [==============================] - 0s 14ms/step - loss: -4.6348 - accuracy: 0.4118
Epoch 10/10
3/3 [==============================] - 0s 6ms/step
Epoch 1/10
9/9 [==============================] - 1s 13m

9/9 [==============================] - 0s 15ms/step - loss: -5.9198 - accuracy: 0.2206
Epoch 12/20
9/9 [==============================] - 0s 15ms/step - loss: -7.9192 - accuracy: 0.3346
Epoch 13/20
9/9 [==============================] - 0s 15ms/step - loss: -8.8770 - accuracy: 0.4228
Epoch 14/20
9/9 [==============================] - 0s 15ms/step - loss: -8.9607 - accuracy: 0.3309
Epoch 15/20
9/9 [==============================] - 0s 15ms/step - loss: -10.2020 - accuracy: 0.4228
Epoch 16/20
9/9 [==============================] - 0s 15ms/step - loss: -11.1508 - accuracy: 0.4081
Epoch 17/20
9/9 [==============================] - 0s 15ms/step - loss: -11.8759 - accuracy: 0.4081
Epoch 18/20
9/9 [==============================] - 0s 16ms/step - loss: -13.1289 - accuracy: 0.4412
Epoch 19/20
9/9 [==============================] - 0s 15ms/step - loss: -13.9256 - accuracy: 0.4522
Epoch 20/20
3/3 [==============================] - 0s 5ms/step
Epoch 1/20
9/9 [==============================] - 1s 

9/9 [==============================] - 0s 17ms/step - loss: -9.7461 - accuracy: 0.3713
Epoch 13/30
9/9 [==============================] - 0s 17ms/step - loss: -10.1368 - accuracy: 0.3971
Epoch 14/30
9/9 [==============================] - 0s 17ms/step - loss: -10.2942 - accuracy: 0.3456
Epoch 15/30
9/9 [==============================] - 0s 17ms/step - loss: -11.8957 - accuracy: 0.3971
Epoch 16/30
9/9 [==============================] - 0s 17ms/step - loss: -12.7834 - accuracy: 0.3824
Epoch 17/30
9/9 [==============================] - 0s 17ms/step - loss: -13.0777 - accuracy: 0.4338
Epoch 18/30
9/9 [==============================] - 0s 17ms/step - loss: -13.8855 - accuracy: 0.4191
Epoch 19/30
9/9 [==============================] - 0s 18ms/step - loss: -13.5744 - accuracy: 0.3787
Epoch 20/30
9/9 [==============================] - 0s 18ms/step - loss: -14.2740 - accuracy: 0.3971
Epoch 21/30
9/9 [==============================] - 0s 18ms/step - loss: -15.2524 - accuracy: 0.4154
Epoch 22/30
9

9/9 [==============================] - 0s 14ms/step - loss: -0.1158 - accuracy: 0.0037
Epoch 4/30
9/9 [==============================] - 0s 15ms/step - loss: -0.9292 - accuracy: 0.1250
Epoch 5/30
9/9 [==============================] - 0s 15ms/step - loss: -2.1296 - accuracy: 0.2904
Epoch 6/30
9/9 [==============================] - 0s 16ms/step - loss: -3.4346 - accuracy: 0.2904
Epoch 7/30
9/9 [==============================] - 0s 15ms/step - loss: -4.1694 - accuracy: 0.3382
Epoch 8/30
9/9 [==============================] - 0s 15ms/step - loss: -5.5296 - accuracy: 0.3603
Epoch 9/30
9/9 [==============================] - 0s 16ms/step - loss: -5.6030 - accuracy: 0.3897
Epoch 10/30
9/9 [==============================] - 0s 16ms/step - loss: -6.6258 - accuracy: 0.3382
Epoch 11/30
9/9 [==============================] - 0s 16ms/step - loss: -7.1339 - accuracy: 0.3971
Epoch 12/30
9/9 [==============================] - 0s 16ms/step - loss: -7.4762 - accuracy: 0.3272
Epoch 13/30
9/9 [===========

5/5 [==============================] - 0s 29ms/step - loss: -0.0164 - accuracy: 0.0110
Epoch 5/10
5/5 [==============================] - 0s 29ms/step - loss: -0.6185 - accuracy: 0.1287
Epoch 6/10
5/5 [==============================] - 0s 29ms/step - loss: -1.5622 - accuracy: 0.2537
Epoch 7/10
5/5 [==============================] - 0s 29ms/step - loss: -2.6117 - accuracy: 0.3419
Epoch 8/10
5/5 [==============================] - 0s 29ms/step - loss: -3.1441 - accuracy: 0.4265
Epoch 9/10
5/5 [==============================] - 0s 29ms/step - loss: -3.4697 - accuracy: 0.2684
Epoch 10/10
3/3 [==============================] - 0s 6ms/step
Epoch 1/10
5/5 [==============================] - 1s 30ms/step - loss: 0.6485 - accuracy: 0.0919
Epoch 2/10
5/5 [==============================] - 0s 30ms/step - loss: 0.3450 - accuracy: 0.0037
Epoch 3/10
5/5 [==============================] - 0s 31ms/step - loss: 0.0531 - accuracy: 0.0037
Epoch 4/10
5/5 [==============================] - 0s 32ms/step - loss

...

5/5 [==============================] - 0s 27ms/step - loss: -9.8094 - accuracy: 0.3750
Epoch 29/30
5/5 [==============================] - 0s 27ms/step - loss: -8.9640 - accuracy: 0.4191
Epoch 30/30
3/3 [==============================] - 0s 5ms/step
Epoch 1/30
5/5 [==============================] - 1s 27ms/step - loss: 0.6958 - accuracy: 0.4706
Epoch 2/30
5/5 [==============================] - 0s 28ms/step - loss: 0.6773 - accuracy: 0.6103
Epoch 3/30
5/5 [==============================] - 0s 29ms/step - loss: 0.6401 - accuracy: 0.6507
Epoch 4/30
5/5 [==============================] - 0s 29ms/step - loss: 0.5656 - accuracy: 0.7022
Epoch 5/30
5/5 [==============================] - 0s 29ms/step - loss: 0.4828 - accuracy: 0.7647
Epoch 6/30
5/5 [==============================] - 0s 30ms/step - loss: 0.4290 - accuracy: 0.7904
Epoch 7/30
5/5 [==============================] - 0s 29ms/step - loss: 0.3276 - accuracy: 0.8603
Epoch 8/30
5/5 [==============================] - 0s 29ms/step - loss: 0

3/3 [==============================] - 0s 35ms/step - loss: 0.4365 - accuracy: 0.8162
Epoch 9/10
3/3 [==============================] - 0s 34ms/step - loss: 0.3770 - accuracy: 0.8493
Epoch 10/10
3/3 [==============================] - 0s 5ms/step
Epoch 1/20
3/3 [==============================] - 1s 31ms/step - loss: 0.6651 - accuracy: 0.1103
Epoch 2/20
3/3 [==============================] - 0s 33ms/step - loss: 0.4807 - accuracy: 0.0037
Epoch 3/20
3/3 [==============================] - 0s 32ms/step - loss: 0.1419 - accuracy: 0.0037
Epoch 4/20
3/3 [==============================] - 0s 32ms/step - loss: -0.1033 - accuracy: 0.0037
Epoch 5/20
3/3 [==============================] - 0s 33ms/step - loss: -0.2178 - accuracy: 0.0037
Epoch 6/20
3/3 [==============================] - 0s 33ms/step - loss: -0.3157 - accuracy: 0.0037
Epoch 7/20
3/3 [==============================] - 0s 33ms/step - loss: -0.6878 - accuracy: 0.0074
Epoch 8/20
3/3 [==============================] - 0s 33ms/step - loss: 

3/3 [==============================] - 0s 34ms/step - loss: -5.6684 - accuracy: 0.3566
Epoch 20/20
3/3 [==============================] - 0s 6ms/step
Epoch 1/20
3/3 [==============================] - 1s 30ms/step - loss: 0.6986 - accuracy: 0.4338
Epoch 2/20
3/3 [==============================] - 0s 30ms/step - loss: 0.6824 - accuracy: 0.6397
Epoch 3/20
3/3 [==============================] - 0s 32ms/step - loss: 0.6648 - accuracy: 0.7096
Epoch 4/20
3/3 [==============================] - 0s 32ms/step - loss: 0.6308 - accuracy: 0.6985
Epoch 5/20
3/3 [==============================] - 0s 32ms/step - loss: 0.5733 - accuracy: 0.7096
Epoch 6/20
3/3 [==============================] - 0s 33ms/step - loss: 0.5417 - accuracy: 0.7132
Epoch 7/20
3/3 [==============================] - 0s 35ms/step - loss: 0.4702 - accuracy: 0.7721
Epoch 8/20
3/3 [==============================] - 0s 34ms/step - loss: 0.4085 - accuracy: 0.8199
Epoch 9/20
3/3 [==============================] - 0s 34ms/step - loss: 0.3

3/3 [==============================] - 0s 5ms/step
Epoch 1/30
3/3 [==============================] - 1s 31ms/step - loss: 0.6854 - accuracy: 0.1066
Epoch 2/30
3/3 [==============================] - 0s 32ms/step - loss: 0.5306 - accuracy: 0.0037
Epoch 3/30
3/3 [==============================] - 0s 32ms/step - loss: 0.2951 - accuracy: 0.0074
Epoch 4/30
3/3 [==============================] - 0s 33ms/step - loss: 0.1277 - accuracy: 0.0037
Epoch 5/30
3/3 [==============================] - 0s 35ms/step - loss: -0.0054 - accuracy: 0.0037
Epoch 6/30
3/3 [==============================] - 0s 35ms/step - loss: -0.3796 - accuracy: 0.0294
Epoch 7/30
3/3 [==============================] - 0s 37ms/step - loss: -0.8882 - accuracy: 0.2059
Epoch 8/30
3/3 [==============================] - 0s 35ms/step - loss: -1.2967 - accuracy: 0.3125
Epoch 9/30
3/3 [==============================] - 0s 35ms/step - loss: -1.8585 - accuracy: 0.2831
Epoch 10/30
3/3 [==============================] - 0s 34ms/step - loss:

3/3 [==============================] - 0s 34ms/step - loss: 0.3195 - accuracy: 0.8603
Epoch 22/30
3/3 [==============================] - 0s 33ms/step - loss: 0.0676 - accuracy: 0.9853
Epoch 23/30
3/3 [==============================] - 0s 33ms/step - loss: 0.1625 - accuracy: 0.9228
Epoch 24/30
3/3 [==============================] - 0s 33ms/step - loss: 0.0561 - accuracy: 0.9853
Epoch 25/30
3/3 [==============================] - 0s 33ms/step - loss: 0.0783 - accuracy: 0.9743
Epoch 26/30
3/3 [==============================] - 0s 34ms/step - loss: 0.0725 - accuracy: 0.9743
Epoch 27/30
3/3 [==============================] - 0s 34ms/step - loss: 0.0253 - accuracy: 0.9963
Epoch 28/30
3/3 [==============================] - 0s 33ms/step - loss: 0.0176 - accuracy: 0.9963
Epoch 29/30
3/3 [==============================] - 0s 33ms/step - loss: 0.0147 - accuracy: 1.0000
Epoch 30/30
3/3 [==============================] - 0s 4ms/step
Epoch 1/20
11/11 [==============================] - 1s 13ms/step - 

In [120]:
# Get the best model
ie_best_model = ie_RNN_grid_search.best_estimator_

results = ie_RNN_grid_search.cv_results_
params = results['params']
mean_test_scores = results['mean_test_score']

# Print the accuracy scores for each combination
for param, score in zip(params, mean_test_scores):
    print("Parameters:", param)
    print("Accuracy:", score)
    print()

print("BEST PARAMS: ", ie_RNN_grid_search.best_params_)

# Evaluate the best model on the test set
test_loss, test_accuracy = best_model.model.evaluate(X_test, y_test)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

Parameters: {'batch_size': 32, 'epochs': 10}
Accuracy: 0.2735294117647059

Parameters: {'batch_size': 32, 'epochs': 20}
Accuracy: 0.3558823529411764

Parameters: {'batch_size': 32, 'epochs': 30}
Accuracy: 0.311764705882353

Parameters: {'batch_size': 64, 'epochs': 10}
Accuracy: 0.3323529411764706

Parameters: {'batch_size': 64, 'epochs': 20}
Accuracy: 0.3411764705882353

Parameters: {'batch_size': 64, 'epochs': 30}
Accuracy: 0.29705882352941176

Parameters: {'batch_size': 128, 'epochs': 10}
Accuracy: 0.24411764705882355

Parameters: {'batch_size': 128, 'epochs': 20}
Accuracy: 0.31176470588235294

Parameters: {'batch_size': 128, 'epochs': 30}
Accuracy: 0.29117647058823526

BEST PARAMS:  {'batch_size': 32, 'epochs': 20}
7/7 [==============================] - 0s 5ms/step - loss: -1.3887 - accuracy: 0.1826
Test Loss: -1.388683557510376
Test Accuracy: 0.1826484054327011


In [112]:
model, X_train, X_test, y_train, y_test = train_model(ukNewsDF)

MAX LENGTH:  17 300
Epoch 1/10


C:\Users\asus\AppData\Local\Temp/ipykernel_27504/3650679903.py:59: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  rnn_model = KerasClassifier(build_fn=create_rnn_model, epochs=10, batch_size=32)


28/28 [==============================] - 1s 12ms/step - loss: -0.0011 - accuracy: 0.0149
Epoch 2/10
28/28 [==============================] - 0s 13ms/step - loss: -2.1678 - accuracy: 0.2509
Epoch 3/10
28/28 [==============================] - 0s 13ms/step - loss: -3.9575 - accuracy: 0.2852
Epoch 4/10
28/28 [==============================] - 0s 13ms/step - loss: -5.6064 - accuracy: 0.2795
Epoch 5/10
28/28 [==============================] - 0s 13ms/step - loss: -6.9315 - accuracy: 0.3517
Epoch 6/10
28/28 [==============================] - 0s 13ms/step - loss: -9.4126 - accuracy: 0.3540
Epoch 7/10
28/28 [==============================] - 0s 13ms/step - loss: -10.6766 - accuracy: 0.3368
Epoch 8/10
28/28 [==============================] - 0s 13ms/step - loss: -12.2790 - accuracy: 0.3711
Epoch 9/10
28/28 [==============================] - 0s 13ms/step - loss: -14.0560 - accuracy: 0.3631
Epoch 10/10
7/7 [==============================] - 0s 7ms/step - loss: -5.6458 - accuracy: 0.3379
Accuracy: 

In [114]:
uk_RNN_grid_search = cross_validate(model, X_train_vec=X_train, y_train=y_train, y_test=y_test, X_test_vec=X_test)

Epoch 1/10
22/22 [==============================] - 1s 12ms/step - loss: 0.0806 - accuracy: 0.0115
Epoch 2/10
22/22 [==============================] - 0s 11ms/step - loss: -1.4181 - accuracy: 0.1504
Epoch 3/10
22/22 [==============================] - 0s 12ms/step - loss: -3.7615 - accuracy: 0.2521
Epoch 4/10
22/22 [==============================] - 0s 12ms/step - loss: -6.1525 - accuracy: 0.3266
Epoch 5/10
22/22 [==============================] - 0s 12ms/step - loss: -7.7225 - accuracy: 0.3395
Epoch 6/10
22/22 [==============================] - 0s 12ms/step - loss: -8.9436 - accuracy: 0.3625
Epoch 7/10
22/22 [==============================] - 0s 12ms/step - loss: -9.9839 - accuracy: 0.3209
Epoch 8/10
22/22 [==============================] - 0s 12ms/step - loss: -11.6865 - accuracy: 0.3725
Epoch 9/10
22/22 [==============================] - 0s 12ms/step - loss: -13.2562 - accuracy: 0.3782
Epoch 10/10
6/6 [==============================] - 0s 5ms/step
Epoch 1/10
22/22 [==================

22/22 [==============================] - 0s 12ms/step - loss: -11.9923 - accuracy: 0.3481
Epoch 10/20
22/22 [==============================] - 0s 12ms/step - loss: -13.3904 - accuracy: 0.3682
Epoch 11/20
22/22 [==============================] - 0s 12ms/step - loss: -14.0654 - accuracy: 0.3711
Epoch 12/20
22/22 [==============================] - 0s 12ms/step - loss: -15.3459 - accuracy: 0.3725
Epoch 13/20
22/22 [==============================] - 0s 12ms/step - loss: -17.1229 - accuracy: 0.3954
Epoch 14/20
22/22 [==============================] - 0s 12ms/step - loss: -19.1049 - accuracy: 0.4126
Epoch 15/20
22/22 [==============================] - 0s 12ms/step - loss: -19.6581 - accuracy: 0.3983
Epoch 16/20
22/22 [==============================] - 0s 12ms/step - loss: -21.9951 - accuracy: 0.3997
Epoch 17/20
22/22 [==============================] - 0s 12ms/step - loss: -23.6306 - accuracy: 0.4255
Epoch 18/20
22/22 [==============================] - 0s 12ms/step - loss: -24.4593 - accuracy:

22/22 [==============================] - 0s 13ms/step - loss: -11.6050 - accuracy: 0.3496
Epoch 9/30
22/22 [==============================] - 0s 13ms/step - loss: -13.3605 - accuracy: 0.3725
Epoch 10/30
22/22 [==============================] - 0s 13ms/step - loss: -16.0563 - accuracy: 0.3997
Epoch 11/30
22/22 [==============================] - 0s 13ms/step - loss: -18.1797 - accuracy: 0.4026
Epoch 12/30
22/22 [==============================] - 0s 13ms/step - loss: -20.7200 - accuracy: 0.4241
Epoch 13/30
22/22 [==============================] - 0s 13ms/step - loss: -21.9995 - accuracy: 0.4312
Epoch 14/30
22/22 [==============================] - 0s 13ms/step - loss: -23.1772 - accuracy: 0.4112
Epoch 15/30
22/22 [==============================] - 0s 13ms/step - loss: -24.6974 - accuracy: 0.4255
Epoch 16/30
22/22 [==============================] - 0s 13ms/step - loss: -26.6001 - accuracy: 0.4312
Epoch 17/30
22/22 [==============================] - 0s 13ms/step - loss: -28.0097 - accuracy: 

22/22 [==============================] - 0s 14ms/step - loss: -35.2303 - accuracy: 0.4441
Epoch 28/30
22/22 [==============================] - 0s 14ms/step - loss: -36.9151 - accuracy: 0.4384
Epoch 29/30
22/22 [==============================] - 0s 13ms/step - loss: -34.5506 - accuracy: 0.4384
Epoch 30/30
6/6 [==============================] - 0s 5ms/step
Epoch 1/30
22/22 [==============================] - 1s 11ms/step - loss: 0.6646 - accuracy: 0.6237
Epoch 2/30
22/22 [==============================] - 0s 12ms/step - loss: 0.5419 - accuracy: 0.7239
Epoch 3/30
22/22 [==============================] - 0s 12ms/step - loss: 0.4323 - accuracy: 0.8011
Epoch 4/30
22/22 [==============================] - 0s 13ms/step - loss: 0.2966 - accuracy: 0.8798
Epoch 5/30
22/22 [==============================] - 0s 13ms/step - loss: 0.2215 - accuracy: 0.9170
Epoch 6/30
22/22 [==============================] - 0s 14ms/step - loss: 0.2065 - accuracy: 0.9242
Epoch 7/30
22/22 [==============================]

11/11 [==============================] - 0s 25ms/step - loss: -4.8794 - accuracy: 0.3152
Epoch 7/10
11/11 [==============================] - 0s 25ms/step - loss: -6.2113 - accuracy: 0.3410
Epoch 8/10
11/11 [==============================] - 0s 24ms/step - loss: -7.4504 - accuracy: 0.3811
Epoch 9/10
11/11 [==============================] - 0s 25ms/step - loss: -8.3718 - accuracy: 0.3954
Epoch 10/10
6/6 [==============================] - 0s 4ms/step
Epoch 1/10
11/11 [==============================] - 1s 22ms/step - loss: 0.3082 - accuracy: 0.0201
Epoch 2/10
11/11 [==============================] - 0s 24ms/step - loss: -0.7712 - accuracy: 0.0860
Epoch 3/10
11/11 [==============================] - 0s 25ms/step - loss: -2.3695 - accuracy: 0.2822
Epoch 4/10
11/11 [==============================] - 0s 24ms/step - loss: -4.0950 - accuracy: 0.3123
Epoch 5/10
11/11 [==============================] - 0s 25ms/step - loss: -5.2148 - accuracy: 0.3596
Epoch 6/10
11/11 [==============================]

...

11/11 [==============================] - 0s 25ms/step - loss: 0.0646 - accuracy: 0.9800
Epoch 14/30
11/11 [==============================] - 0s 25ms/step - loss: 0.0499 - accuracy: 0.9843
Epoch 15/30
11/11 [==============================] - 0s 24ms/step - loss: 0.0387 - accuracy: 0.9886
Epoch 16/30
11/11 [==============================] - 0s 24ms/step - loss: 0.0242 - accuracy: 0.9914
Epoch 17/30
11/11 [==============================] - 0s 24ms/step - loss: 0.0245 - accuracy: 0.9928
Epoch 18/30
11/11 [==============================] - 0s 25ms/step - loss: 0.0112 - accuracy: 0.9957
Epoch 19/30
11/11 [==============================] - 0s 25ms/step - loss: 0.0210 - accuracy: 0.9957
Epoch 20/30
11/11 [==============================] - 0s 25ms/step - loss: 0.0126 - accuracy: 0.9986
Epoch 21/30
11/11 [==============================] - 0s 24ms/step - loss: 0.0093 - accuracy: 0.9986
Epoch 22/30
11/11 [==============================] - 0s 24ms/step - loss: 0.0044 - accuracy: 0.9986
Epoch 23/30


6/6 [==============================] - 0s 41ms/step - loss: 0.5878 - accuracy: 0.7196
Epoch 3/10
6/6 [==============================] - 0s 42ms/step - loss: 0.5387 - accuracy: 0.7196
Epoch 4/10
6/6 [==============================] - 0s 40ms/step - loss: 0.4736 - accuracy: 0.7682
Epoch 5/10
6/6 [==============================] - 0s 41ms/step - loss: 0.4282 - accuracy: 0.8140
Epoch 6/10
6/6 [==============================] - 0s 40ms/step - loss: 0.3530 - accuracy: 0.8426
Epoch 7/10
6/6 [==============================] - 0s 40ms/step - loss: 0.2636 - accuracy: 0.9013
Epoch 8/10
6/6 [==============================] - 0s 41ms/step - loss: 0.1948 - accuracy: 0.9328
Epoch 9/10
6/6 [==============================] - 0s 41ms/step - loss: 0.1258 - accuracy: 0.9571
Epoch 10/10
6/6 [==============================] - 0s 5ms/step
Epoch 1/10
6/6 [==============================] - 1s 35ms/step - loss: 0.5546 - accuracy: 0.0658
Epoch 2/10
6/6 [==============================] - 0s 37ms/step - loss: 0.04

6/6 [==============================] - 0s 38ms/step - loss: 0.6171 - accuracy: 0.7110
Epoch 3/20
6/6 [==============================] - 0s 39ms/step - loss: 0.5411 - accuracy: 0.7268
Epoch 4/20
6/6 [==============================] - 0s 39ms/step - loss: 0.4812 - accuracy: 0.7654
Epoch 5/20
6/6 [==============================] - 0s 40ms/step - loss: 0.4311 - accuracy: 0.8069
Epoch 6/20
6/6 [==============================] - 0s 40ms/step - loss: 0.3570 - accuracy: 0.8355
Epoch 7/20
6/6 [==============================] - 0s 40ms/step - loss: 0.2955 - accuracy: 0.8784
Epoch 8/20
6/6 [==============================] - 0s 40ms/step - loss: 0.2655 - accuracy: 0.8956
Epoch 9/20
6/6 [==============================] - 0s 40ms/step - loss: 0.1729 - accuracy: 0.9442
Epoch 10/20
6/6 [==============================] - 0s 40ms/step - loss: 0.1170 - accuracy: 0.9700
Epoch 11/20
6/6 [==============================] - 0s 42ms/step - loss: 0.0679 - accuracy: 0.9857
Epoch 12/20
6/6 [======================

6/6 [==============================] - 0s 38ms/step - loss: -7.6871 - accuracy: 0.3968
Epoch 14/30
6/6 [==============================] - 0s 38ms/step - loss: -8.0140 - accuracy: 0.3940
Epoch 15/30
6/6 [==============================] - 0s 39ms/step - loss: -8.5219 - accuracy: 0.3596
Epoch 16/30
6/6 [==============================] - 0s 39ms/step - loss: -8.9800 - accuracy: 0.3968
Epoch 17/30
6/6 [==============================] - 0s 38ms/step - loss: -9.1853 - accuracy: 0.3840
Epoch 18/30
6/6 [==============================] - 0s 38ms/step - loss: -10.1182 - accuracy: 0.4155
Epoch 19/30
6/6 [==============================] - 0s 38ms/step - loss: -10.1314 - accuracy: 0.3797
Epoch 20/30
6/6 [==============================] - 0s 38ms/step - loss: -10.8547 - accuracy: 0.3883
Epoch 21/30
6/6 [==============================] - 0s 38ms/step - loss: -11.3356 - accuracy: 0.4269
Epoch 22/30
6/6 [==============================] - 0s 38ms/step - loss: -11.6763 - accuracy: 0.3940
Epoch 23/30
6/6 [

6/6 [==============================] - 0s 39ms/step - loss: -1.0202 - accuracy: 0.1645
Epoch 5/30
6/6 [==============================] - 0s 38ms/step - loss: -1.8277 - accuracy: 0.1645
Epoch 6/30
6/6 [==============================] - 0s 41ms/step - loss: -2.8086 - accuracy: 0.2861
Epoch 7/30
6/6 [==============================] - 0s 39ms/step - loss: -3.8420 - accuracy: 0.2990
Epoch 8/30
6/6 [==============================] - 0s 39ms/step - loss: -4.4245 - accuracy: 0.2546
Epoch 9/30
6/6 [==============================] - 0s 39ms/step - loss: -5.1179 - accuracy: 0.3205
Epoch 10/30
6/6 [==============================] - 0s 39ms/step - loss: -5.6483 - accuracy: 0.3333
Epoch 11/30
6/6 [==============================] - 0s 39ms/step - loss: -6.1400 - accuracy: 0.3948
Epoch 12/30
6/6 [==============================] - 0s 39ms/step - loss: -6.8357 - accuracy: 0.3348
Epoch 13/30
6/6 [==============================] - 0s 39ms/step - loss: -7.2628 - accuracy: 0.3977
Epoch 14/30
6/6 [==========

In [119]:
# Get the best model
uk_best_model = uk_RNN_grid_search.best_estimator_

results = uk_RNN_grid_search.cv_results_
params = results['params']
mean_test_scores = results['mean_test_score']

# Print the accuracy scores for each combination
for param, score in zip(params, mean_test_scores):
    print("Parameters:", param)
    print("Accuracy:", score)
    print()

print("BEST PARAMS: ", uk_RNN_grid_search.best_params_)

# Evaluate the best model on the test set
test_loss, test_accuracy = best_model.model.evaluate(X_test, y_test)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

Parameters: {'batch_size': 32, 'epochs': 10}
Accuracy: 0.3543776683087028

Parameters: {'batch_size': 32, 'epochs': 20}
Accuracy: 0.39776026272577997

Parameters: {'batch_size': 32, 'epochs': 30}
Accuracy: 0.40122824302134646

Parameters: {'batch_size': 64, 'epochs': 10}
Accuracy: 0.3908440065681445

Parameters: {'batch_size': 64, 'epochs': 20}
Accuracy: 0.37372742200328407

Parameters: {'batch_size': 64, 'epochs': 30}
Accuracy: 0.4103054187192118

Parameters: {'batch_size': 128, 'epochs': 10}
Accuracy: 0.3898390804597701

Parameters: {'batch_size': 128, 'epochs': 20}
Accuracy: 0.39783251231527095

Parameters: {'batch_size': 128, 'epochs': 30}
Accuracy: 0.3978259441707718

BEST PARAMS:  {'batch_size': 64, 'epochs': 30}
7/7 [==============================] - 0s 5ms/step - loss: -1.3887 - accuracy: 0.1826
Test Loss: -1.388683557510376
Test Accuracy: 0.1826484054327011


## Cross Validate of the RNN models

In [132]:
cross_validate(irelandNewsDF, ie_best_model)

C:\Users\asus\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


MAX LENGTH:  21 300
Epoch 1/20
11/11 [==============================] - 1s 13ms/step - loss: 0.6878 - accuracy: 0.5559
Epoch 2/20
11/11 [==============================] - 0s 13ms/step - loss: 0.6130 - accuracy: 0.7118
Epoch 3/20
11/11 [==============================] - 0s 13ms/step - loss: 0.5023 - accuracy: 0.7824
Epoch 4/20
11/11 [==============================] - 0s 13ms/step - loss: 0.4220 - accuracy: 0.7824
Epoch 5/20
11/11 [==============================] - 0s 13ms/step - loss: 0.2962 - accuracy: 0.8618
Epoch 6/20
11/11 [==============================] - 0s 13ms/step - loss: 0.1688 - accuracy: 0.9324
Epoch 7/20
11/11 [==============================] - 0s 15ms/step - loss: 0.0649 - accuracy: 0.9824
Epoch 8/20
11/11 [==============================] - 0s 15ms/step - loss: 0.1416 - accuracy: 0.9500
Epoch 9/20
11/11 [==============================] - 0s 15ms/step - loss: 0.1345 - accuracy: 0.9588
Epoch 10/20
11/11 [==============================] - 0s 15ms/step - loss: 0.0374 - accura

11/11 [==============================] - 0s 16ms/step - loss: -10.1922 - accuracy: 0.4164
Epoch 19/20
11/11 [==============================] - 0s 16ms/step - loss: -10.9791 - accuracy: 0.4340
Epoch 20/20
3/3 [==============================] - 0s 7ms/step - loss: -4.7895 - accuracy: 0.3176
Accuracy: 0.3176470696926117
MAX LENGTH:  20 300
Epoch 1/20
11/11 [==============================] - 1s 13ms/step - loss: 0.4242 - accuracy: 0.0205
Epoch 2/20
11/11 [==============================] - 0s 13ms/step - loss: 0.0825 - accuracy: 0.0029
Epoch 3/20
11/11 [==============================] - 0s 12ms/step - loss: -0.6929 - accuracy: 0.0850
Epoch 4/20
11/11 [==============================] - 0s 13ms/step - loss: -1.9386 - accuracy: 0.2493
Epoch 5/20
11/11 [==============================] - 0s 13ms/step - loss: -3.7550 - accuracy: 0.3578
Epoch 6/20
11/11 [==============================] - 0s 13ms/step - loss: -4.8489 - accuracy: 0.3460
Epoch 7/20
11/11 [==============================] - 0s 13ms/ste

In [133]:
cross_validate(ukNewsDF, uk_best_model)

MAX LENGTH:  17 300


C:\Users\asus\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Epoch 1/30
14/14 [==============================] - 1s 23ms/step - loss: 0.3239 - accuracy: 0.0229
Epoch 2/30
14/14 [==============================] - 0s 24ms/step - loss: -0.5109 - accuracy: 0.0401
Epoch 3/30
14/14 [==============================] - 0s 24ms/step - loss: -2.0180 - accuracy: 0.2509
Epoch 4/30
14/14 [==============================] - 0s 24ms/step - loss: -3.2978 - accuracy: 0.2646
Epoch 5/30
14/14 [==============================] - 0s 24ms/step - loss: -4.4987 - accuracy: 0.2795
Epoch 6/30
14/14 [==============================] - 0s 24ms/step - loss: -5.7128 - accuracy: 0.3505
Epoch 7/30
14/14 [==============================] - 0s 24ms/step - loss: -7.2843 - accuracy: 0.3150
Epoch 8/30
14/14 [==============================] - 0s 24ms/step - loss: -8.9399 - accuracy: 0.3666
Epoch 9/30
14/14 [==============================] - 0s 24ms/step - loss: -10.1451 - accuracy: 0.4021
Epoch 10/30
14/14 [==============================] - 0s 25ms/step - loss: -10.5935 - accuracy: 0.381

14/14 [==============================] - 0s 27ms/step - loss: -19.0828 - accuracy: 0.4233
Epoch 20/30
14/14 [==============================] - 0s 27ms/step - loss: -18.7177 - accuracy: 0.4130
Epoch 21/30
14/14 [==============================] - 0s 27ms/step - loss: -18.4071 - accuracy: 0.3192
Epoch 22/30
14/14 [==============================] - 0s 27ms/step - loss: -20.2491 - accuracy: 0.3810
Epoch 23/30
14/14 [==============================] - 0s 26ms/step - loss: -21.5732 - accuracy: 0.4176
Epoch 24/30
14/14 [==============================] - 0s 27ms/step - loss: -22.5924 - accuracy: 0.3936
Epoch 25/30
14/14 [==============================] - 0s 27ms/step - loss: -23.9268 - accuracy: 0.4096
Epoch 26/30
14/14 [==============================] - 0s 27ms/step - loss: -24.6336 - accuracy: 0.4222
Epoch 27/30
14/14 [==============================] - 0s 26ms/step - loss: -25.7811 - accuracy: 0.4016
Epoch 28/30
14/14 [==============================] - 0s 26ms/step - loss: -26.5841 - accuracy: